# ARIMAモデルによる時系列予測

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tatsuki-washimi/gwexpy/blob/main/docs/web/en/user_guide/tutorials/advanced_arima.ipynb)

このノートブックでは、`gwexpy` の `TimeSeries` クラスに追加された ARIMA 関連の機能（AR, MA, ARMA, ARIMA/SARIMAX）を使用して、時系列データのモデリングと未来予測を行う方法を解説します。

## 準備

必要なライブラリをインポートします。

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import sklearn.utils.validation

# Monkeypatch for pmdarima compatibility with scikit-learn >= 1.6
_orig_check_array = sklearn.utils.validation.check_array


def _patched_check_array(*args, **kwargs):
    kwargs.pop("force_all_finite", None)
    return _orig_check_array(*args, **kwargs)


sklearn.utils.validation.check_array = _patched_check_array

warnings.filterwarnings(
    "ignore", message=r".*force_all_finite.*", category=FutureWarning
)
warnings.filterwarnings("ignore", category=FutureWarning, module=r"sklearn\\..*")
warnings.filterwarnings("ignore", category=FutureWarning, module=r"pmdarima\\..*")

from gwexpy.timeseries import TimeSeries

## サンプルデータの作成

ARIMAモデルの効果を確認するために、トレンド、季節性（周期性）、ノイズを含んだ模擬データを作成します。

In [ ]:
# Trend + Seasonality + Noise
t0 = 0
dt = 0.1
duration = 30
t = np.arange(0, duration, dt)
n_samples = len(t)

# 1. Linear Trend
trend = 0.3 * t

# 2. Periodic component (Sine)
seasonality = 2.0 * np.sin(2 * np.pi * 0.2 * t)

# 3. Noise
np.random.seed(42)
noise = np.random.normal(0, 0.8, n_samples)

data = trend + seasonality + noise

# Create TimeSeries object
ts = TimeSeries(data, t0=t0, dt=dt, unit="V", name="Sample Data")

ts.plot()
plt.title("Original Data (Trend + Seasonality + Noise)")
plt.show()

## 1. AR (AutoRegressive) モデル

ARモデルは、現在の値が過去の値に依存すると仮定するモデルです。
`ts.ar(p)` メソッドを使用します。`p` は次数です。

In [ ]:
# Fit with order p=3
model_ar = ts.ar(p=3)

# Display model summary
# print(model_ar.summary())

# Plot results
# forecast_steps specifies how many steps to forecast into the future
model_ar.plot(forecast_steps=50, alpha=0.5)
plt.title("AR(3) Model Forecast")
plt.show()

## 2. MA (Moving Average) モデル

MAモデルは、現在の値が過去の予測誤差（ホワイトノイズ）に依存すると仮定するモデルです。
`ts.ma(q)` メソッドを使用します。

In [ ]:
# Fit with order q=3
model_ma = ts.ma(q=3)

# Plot results
model_ma.plot(forecast_steps=50, alpha=0.5)
plt.title("MA(3) Model Forecast")
plt.show()

## 3. ARMA (AutoRegressive Moving Average) モデル

ARとMAを組み合わせたモデルです。
`ts.arma(p, q)` メソッドを使用します。

In [ ]:
# Fit with p=2, q=2
model_arma = ts.arma(p=2, q=2)

model_arma.plot(forecast_steps=50, alpha=0.5)
plt.title("ARMA(2, 2) Model Forecast")
plt.show()

## 4. ARIMA / Auto-ARIMA

ARIMAモデルは、差分を取ることで非定常性（トレンドなど）を除去し、ARMAモデルを適用します。

`ts.arima()` メソッドを使用します。
`auto=True` オプションを指定すると、`pmdarima` ライブラリを使用して最適なパラメータ (p, d, q) を自動的に探索します。これが最も推奨される方法です。

In [ ]:
try:
    print("Searching for optimal ARIMA parameters... (this may take a few seconds)")

    # Seasonality can also be considered
    # m is the number of steps in a seasonal period
    model_auto = ts.arima(auto=True, auto_kwargs={"seasonal": True, "m": 50})

    # Show info of the optimal model found
    print(model_auto.summary())

    # Plot results (Forecasting for a longer period)
    ax = model_auto.plot(forecast_steps=100, alpha=0.5)
    plt.title("Auto-ARIMA Model Forecast (Seasonal)")
    plt.show()
except ImportError as e:
    print(f"Auto-ARIMA skipping: {e}")


## 5. 予測データの取得

プロットだけでなく、`.forecast()` メソッドを使用して予測値の `TimeSeries` オブジェクトを直接取得できます。
これには信頼区間（Confidence Interval）も含まれます。

In [ ]:
steps = 50

# Get forecast values and confidence intervals
forecast_ts, intervals = model_auto.forecast(
    steps=steps, alpha=0.5
) # 50% confidence interval

print("Forecast Data:")
print(forecast_ts)

print("\nConfidence Interval (Upper):")
print(intervals["upper"])

# Forecasting data can be saved or used for other analysis
# forecast_ts.write("forecast.h5")

## 6. 重力波データ解析における応用: ARIMAによるノイズ除去 (Whitening)

重力波データ解析では、定常的な「有色ノイズ」の中から突発的なバースト信号やリングダウン波形を抽出するために、**白色化 (Whitening)** という処理が行われます。
ARIMAモデルを背景ノイズの予測モデルとして学習させ、実測値からその予測値を引く（残差をとる）ことで、ノイズを除去し信号を強調することが可能です。

### Step 1: 有色ノイズの生成と信号の注入
まず、低周波成分が強い有色ノイズ（AR(1)プロセス）を作成し、そこにそのままでは判別しにくい微弱な信号（サインガウシアン）を埋め込みます。
※単位には Astropy で認識可能な "strain" を使用します。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from gwexpy.timeseries import TimeSeries


def generate_mock_data(duration=10, fs=1024):
    t = np.linspace(0, duration, int(duration * fs))

    # 1. Background Noise (AR(1) process with strong autocorrelation)
    np.random.seed(42)
    white_noise = np.random.normal(0, 1, len(t))
    colored_noise = np.zeros_like(white_noise)
    for i in range(1, len(t)):
        colored_noise[i] = 0.95 * colored_noise[i - 1] + white_noise[i]

    # 2. Injected Signal (Sine-Gaussian)
    center_time = 5.0
    sig_freq = 50.0
    width = 0.1
    signal = (
        2.0
        * np.exp(-((t - center_time) ** 2) / (2 * width**2))
        * np.sin(2 * np.pi * sig_freq * t)
    )

    data = colored_noise + signal
    # Use a standard unit string that astropy/gwpy can recognize
    return TimeSeries(data, t0=0, dt=1 / fs, unit="strain", name="MockData")


ts = generate_mock_data()

plt.figure(figsize=(10, 4))
plt.plot(ts, color="gray")
plt.title("Original Noisy Data (Signal is hidden here)")
plt.xlabel("Time [s]")
plt.grid(True, linestyle=":")
plt.show()

### Step 2: ARIMAによるノイズモデリング
`auto=True` を使って、背景ノイズの自己相関構造を表現する最適なモデルを探索します。

In [ ]:
print("Searching for optimal noise model parameters...")
# We prioritize AR model for whitening, so we set max_p=5 and max_q=0.
model = ts.arima(auto=True, max_p=5, max_q=0)
print(f"Best model order: {model.res.model.order}")
print(model.summary())

### Step 3: 残差（Residuals）の抽出と可視化
モデルが予測した「定常ノイズ」を引くことで、残差 $x_r = x - \hat{x}$ を求めます。これが白色化されたデータとなります。

In [ ]:
ts_whitened = model.residuals()
ts_whitened.name = "Whitened Data"

fig, ax = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

ax[0].plot(ts, color="gray", alpha=0.5, label="Original")
ax[0].set_title("Original Noisy Data")
ax[0].legend(loc="upper right")

ax[1].plot(ts_whitened, color="red", label="Residuals (Whitened)")
ax[1].set_title("Whitened Data (Signal is now visible!)")
ax[1].axvspan(4.8, 5.2, color="orange", alpha=0.3, label="Injected Signal Position")
ax[1].legend(loc="upper right")

plt.tight_layout()
plt.show()

### Step 4: 周波数領域での評価
パワースペクトル密度 (PSD) を計算して比較します。

In [ ]:
fft_original = ts.psd()
fft_whitened = ts_whitened.psd()

plt.figure(figsize=(10, 5))
plt.loglog(
    fft_original.frequencies.value,
    fft_original.value,
    label="Original Noisy PSD",
    alpha=0.7,
)
plt.loglog(
    fft_whitened.frequencies.value,
    fft_whitened.value,
    label="Whitened PSD (Residuals)",
    alpha=0.7,
)
plt.title("PSD Comparison: Effect of ARIMA Whitening")
plt.xlabel("Frequency [Hz]")
plt.ylabel("PSD [1/Hz]")
plt.legend()
plt.grid(True, which="both", linestyle=":")
plt.show()

### まとめ
- `model.residuals()` を用いることで、複雑な数式を書かずに簡単に白色化処理が実行できます。
- この手法は、重力波解析におけるバースト信号の探索や、イベント直後のリングダウン波形（指数減衰する正弦波）の抽出に非常に強力です。